In [2]:
from qiskit import *
import numpy as np

# 1. Approach to the problem
The main part of the ‘false asteroids’ problem is to find a way to identify some universal characteristic of all the boards that cannot be cleared with 3 laser shots. We need to first ask the question, what is the minimum number of asteroids that is required to make the board unsolvable? We know that boards with only 3 asteroids are clearable because we have 3 laser shots. Looking at boards with 4 asteroids we see that the diagonal board cannot be cleared with 3 laser shots (1 represents presence of an asteroid at that point):<br>
1 0 0 0<br>
0 1 0 0<br>
0 0 1 0<br>
0 0 0 1<br>
In fact, all permutations of the diagonal board cannot be cleared in 3 laser shots. By permutations what I mean is that we exchange the row numbers of any 2 asteroids while keeping the column numbers the same, some examples are given below.<br>
0 1 0 0 &emsp;  0 1 0 0 &emsp;  0 0 1 0<br>
1 0 0 0	&emsp;	1 0 0 0	&emsp;	0 1 0 0<br>
0 0 1 0	&emsp;	0 0 0 1	&emsp;	1 0 0 0<br>
0 0 0 1	&emsp;	0 0 1 0	&emsp;	0 0 0 1<br>
There will be 4! (=24) such boards. If any of the boards that we are given contain these boards then it cannot be cleared. In more mathematical terms, all matrices that can be cleared have permanent equal to 0. The permanent is a property similar to the determinant, please check it out on Wikipedia if you are not familiar with it. All the boards with 4 asteroids shown above have permanent equal to 1. When you have six asteroids it is possible to have matrices with permanent equal to 0, 1 and 2 while only matrices with permanent equal to 0 can be cleared. My approach was to first convert matrices with permanent equal to 2 to matrices with permanent equal to 1 (I will explain how later). Then all matrices with permanent equal to 1 were phase flipped (pi-phase) as is done in Grover’s algorithm.
# 2. QRAM implementation
The boards' data is loaded using a QRAM structure with 4 address qubits, 16 data qubits and a total of 3 ancillary qubits. The order of 'writes' is a 4-bit Gray code sequence starting from 1111, i.e.,<br>
    1111 → 1110 → 1010→1011 →1001→1000→0000→ 0001→0011→ 0010→0110→0111→0101→0100→1100→1101<br>
This way, only one-bit changes between subsequent writes, which minimizes the un-computations that is necessary between different addresses.
# 3. Oracle
My oracle checks for the presence of one of the 24 boards with 4 asteroids and permanent equal to 1 and applies a phase flip if present. However, there is a possibility of the 6-asteroid boards to have permanent equal to 2, for example:<br> 
1 1 0 0<br>
1 1 0 0<br>
0 0 1 0<br>
0 0 0 1<br>
which would trigger the phase flip twice corresponding to the two boards:<br>
1 0 0 0&emsp;		0 1 0 0<br>
0 1 0 0&emsp;		1 0 0 0<br>
0 0 1 0&emsp;		0 0 1 0<br>
0 0 0 1&emsp;		0 0 0 1<br>
We account for this by converting all matrices with permanent equal to 2 to matrices with permanent equal to 1.
#  4. Converting permanent=2 to permanent=1
We carry out this step inside the QRAM implementation while initializing the data qubits. We see that the above example of a matrix with permanent equal to 2:<br>
1 1 0 0<br>
1 1 0 0<br>
0 0 1 0<br>
0 0 0 1<br>
becomes a matrix with permanent equal to 1 if we simply don't initialize one of the four 1's:<br>
1 0 0 0&emsp;		0 1 0 0&emsp;		0 0 0 0&emsp;		0 0 0 0<br>
0 0 0 0&emsp;		0 0 0 0&emsp;		1 0 0 0&emsp;		0 1 0 0<br>
0 0 0 0&emsp;		0 0 0 0&emsp;		0 0 0 0&emsp;		0 0 0 0<br>
0 0 0 0&emsp;		0 0 0 0&emsp;		0 0 0 0&emsp;		0 0 0 0<br>
This can be achieved by simply making use of some if-else conditions, but need to be done carefully to ensure that matrices with permanent =1 initially do not get converted to matrices with permanent =0.
# 5. The Whole circuit
The complete circuit is then composed of:<br>
&emsp;1.Initialization<br>
&emsp;2.QRAM <br>
&emsp;3.Oracle <br>
&emsp;4.Inverse QRAM<br> 
&emsp;5.Diffusion<br>
Inverse QRAM is just QRAM with the order of operations reversed. Qubit 0-3 are the address qubits and 4-19 are data qubits the rest are ancilla qubits which assist in various operations.


In [ ]:
problem_set = \
    [[['0', '2'], ['1', '0'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '3']],
    [['0', '0'], ['1', '1'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
    [['0', '2'], ['1', '0'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '1'], ['3', '3']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '3'], ['1', '2'], ['2', '2'], ['2', '3'], ['3', '0']],
    [['0', '3'], ['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '3'], ['2', '1'], ['2', '3'], ['3', '0']],
    [['0', '1'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '2']],
    [['0', '0'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '3'], ['3', '1']],
    [['0', '1'], ['0', '2'], ['1', '0'], ['1', '2'], ['2', '2'], ['2', '3']],
    [['0', '3'], ['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '0']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '3'], ['3', '0'], ['3', '1']],
    [['0', '1'], ['1', '0'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '1']]]
def week3_ans_func(problem_set):
    ##### build your quantum circuit here
    ##### In addition, please make it a function that can solve the problem even with different inputs (problem_set). We do validation with different inputs. 
    qc=QuantumCircuit(27,4)
    k1=[[1,1],[1,0],[0,0],[0,1]]
    k2=[[0,1],[0,0],[1,0],[1,1]]
    qc.h([0,1,2,3])
    qc.ccx(0,1,25)
    qc.ccx(2,3,26)
    qc.ccx(25,26,24)
    rc=np.zeros((16,8))
    for i in k1: # qram operation
        if i[0]==1:
            if i[1]==0:
                qc.cx(0,25)
                qc.ccx(0,26,24)
        if i[0]==0:
            if i[1]==0:
                qc.x(1)
                qc.cx(1,25)
                qc.ccx(1,26,24) 
                qc.x(1)
        if i[0]==0:
            if i[1]==1:
                qc.x(0)
                qc.cx(0,25)
                qc.ccx(0,26,24)
                qc.x(0)
        for j in k1: 
            if j[0]==1:
                if j[1]==0:
                    qc.ccx(2,25,24)
            if j[0]==0:
                if j[1]==0:
                    qc.x(3)
                    qc.ccx(3,25,24)
                    qc.x(3) 
            if j[0]==0:
                if j[1]==1:
                    qc.x(2)
                    qc.ccx(2,25,24)
                    qc.x(2)
            i0=int(i[0])
            i1=int(i[1])
            j0=int(j[0])
            j1=int(j[1])
            c=0
            for m in range(6):
                k=problem_set[i0*8+i1*4+j0*2+j1][m]
                k00=int(k[0])
                k11=int(k[1])
                if (rc[i0*8+i1*4+j0*2+j1][k00]==0 or rc[i0*8+i1*4+j0*2+j1][k11+4]==0) or c==0:
                    qc.cx(24,k00*4+k11+4)
                    rc[i0*8+i1*4+j0*2+j1][k00]+=1
                    rc[i0*8+i1*4+j0*2+j1][k11+4]+=1
                if c==0 and (rc[i0*8+i1*4+j0*2+j1][k00]==2 or rc[i0*8+i1*4+j0*2+j1][k11+4]==2):
                    for n in range(m+1):
                        l=problem_set[i0*8+i1*4+j0*2+j1][n]
                        l00=int(l[0])
                        l11=int(l[1])
                        if (rc[i0*8+i1*4+j0*2+j1][l00]>1 and rc[i0*8+i1*4+j0*2+j1][l11+4]>1):
                            c+=1
                        
            if j[0]==0:
                if j[1]==1:
                    qc.ccx(3,25,24)
                    
    p=[[0,1,2,3],[0,2,1,3],[0,3,1,2]]#oracle
    for q in p:
        qc.ccx([q[0]+4,q[1]+4,q[2]+12,q[3]+12],[q[1]+8,q[0]+8,q[3]+16,q[2]+16],[20,21,22,23])
        qc.cz([20,21,20,21],[22,22,23,23])
        qc.ccx([q[0]+4,q[1]+4,q[2]+12,q[3]+12],[q[1]+8,q[0]+8,q[3]+16,q[2]+16],[20,21,22,23])
        qc.ccx([q[2]+4,q[3]+4,q[0]+12,q[1]+12],[q[3]+8,q[2]+8,q[1]+16,q[0]+16],[20,21,22,23])
        qc.cz([20,21,20,21],[22,22,23,23])
        qc.ccx([q[2]+4,q[3]+4,q[0]+12,q[1]+12],[q[3]+8,q[2]+8,q[1]+16,q[0]+16],[20,21,22,23])
        
    rc=np.zeros((16,8)) 

    for i in k2: # inverse_qram operation
        for j in k2:
            if j[0]==0:
                if j[1]==1:
                    qc.ccx(3,25,24)
            i0=int(i[0])
            i1=int(i[1])
            j0=int(j[0])
            j1=int(j[1])
            c=0
            for m in range(6):
                k=problem_set[i0*8+i1*4+j0*2+j1][m]
                k00=int(k[0])
                k11=int(k[1])
                if (rc[i0*8+i1*4+j0*2+j1][k00]==0 or rc[i0*8+i1*4+j0*2+j1][k11+4]==0) or c==0:
                    qc.cx(24,k00*4+k11+4)
                    rc[i0*8+i1*4+j0*2+j1][k00]+=1
                    rc[i0*8+i1*4+j0*2+j1][k11+4]+=1
                if c==0 and (rc[i0*8+i1*4+j0*2+j1][k00]==2 or rc[i0*8+i1*4+j0*2+j1][k11+4]==2):
                    for n in range(m+1):
                        l=problem_set[i0*8+i1*4+j0*2+j1][n]
                        l00=int(l[0])
                        l11=int(l[1])
                        if (rc[i0*8+i1*4+j0*2+j1][l00]>1 and rc[i0*8+i1*4+j0*2+j1][l11+4]>1):
                            c+=1

            if j[0]==1:
                if j[1]==0:
                    qc.ccx(2,25,24)
            if j[0]==0:
                if j[1]==0:
                    qc.x(3)
                    qc.ccx(3,25,24)
                    qc.x(3) 
            if j[0]==0:
                if j[1]==1:
                    qc.x(2)
                    qc.ccx(2,25,24)
                    qc.x(2)
        if i[0]==1:
            if i[1]==0:
                qc.cx(0,25)
                qc.ccx(0,26,24)
        if i[0]==0:
            if i[1]==0:
                qc.x(1)
                qc.cx(1,25)
                qc.ccx(1,26,24) 
                qc.x(1)
        if i[0]==0:
            if i[1]==1:
                qc.x(0)
                qc.cx(0,25)
                qc.ccx(0,26,24)
                qc.x(0)
    qc.ccx(25,26,24)
    qc.ccx(0,1,25)
    qc.ccx(2,3,26)


    for k in range(4):
        qc.h(k)
        qc.x(k)
    qc.ccx(0,1,24)
    qc.ccx(2,3,25)
    qc.cz(24,25)
    qc.ccx(2,3,25)
    qc.ccx(0,1,24)
    for k in range(4):
        qc.x(k)
        qc.h(k)

    for a in range(4):
        qc.measure(a,a)
    #### Code for Grover's algorithm with iterations = 1 will be as follows. 
    #### for i in range(1):
    ####   oracle()
    ####   diffusion()
    
    return qc